本文内容主要来自：
- https://blog.csdn.net/wuichuan/article/details/66969315
- https://blog.csdn.net/Hungryof/article/details/71436529
- 忘了一个

### 判定使用GPU
下载了对应的GPU版本的Pytorch之后，要确保GPU是可以进行使用的，通过torch.cuda.is_available()的返回值来进行判断。返回 True 则具有能够使用的GPU。<br>
通过 torch.cuda.device_count() 可以获得能够使用的GPU数量。

### 数据迁移

不论是什么类型的Tensor（FloatTensor或者是LongTensor等等），一律直接使用方法.cuda()即可。<br>
如果要将显存中的数据复制到内存中，则对cuda数据类型使用 .cpu() 方法即可
```
ten1 = torch.FloatTensor(2)
>>>>  6.1101e+24
 	  4.5659e-41
	  [torch.FloatTensor of size 2]
	  
ten1_cuda = ten1.cuda()
>>>>   6.1101e+24
	   4.5659e-41
	   [torch.cuda.FloatTensor of size 2 (GPU 0)]
```

Variable数据的迁移应该特别注意，因为grad_func的原因，如果直接迁移Variable数据可能导致，requires_grad参数被重置为零，所以一般都是先迁移到GPU上之后，在用Varaible进行包装。

### 模型迁移

所谓的模型，其实也就是对输入参数做了一些基本的矩阵运算。所以我们对`模型 .cuda() 实际上也相当于将模型使用到的参数存储到了显存上去。
```
linear = torch.nn.Linear(2, 2)
>>>> Linear(in_features=2, out_features=2)

linear_cuda = linear.cuda()
>>>> Linear(in_features=2, out_features=2)

linear.weight
>>>> Parameter containing:
    -0.6847  0.2149
    -0.5473  0.6863
    [torch.FloatTensor of size 2x2]

linear_cuda.weight
>>>> Parameter containing:
    -0.6847  0.2149
    -0.5473  0.6863
    [torch.cuda.FloatTensor of size 2x2 (GPU 0)]
```

### 多GPU运算

nn.DataParallel

### 例子

(为测试)
准备数据
```
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import  Variable
import torch.nn as nn
x = np.random.randn(1000, 1)*4
w = np.array([0.5,])
bias = -1.68

y_true = np.dot(x, w) + bias  #真实数据
y = y_true + np.random.randn(x.shape[0])#加噪声的数据
#我们需要使用x和y，以及y_true回归出w和bias
```
定义回归网络的类
```
class LinearRression(nn.Module):
    def __init__(self, input_size, out_size):
        super(LinearRression, self).__init__()
        self.x2o = nn.Linear(input_size, out_size)
    #初始化
    def forward(self, x):
        return self.x2o(x)
    #前向传递
```
接下来介绍将定义模型和优化器
```
batch_size = 10
model = LinearRression(1, 1)#回归模型
criterion = nn.MSELoss()  #损失函数
#调用cuda
model.cuda()
criterion.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
losses = []
```
下面就是训（练）练（丹）了
```
for i in range(epoches):
    loss = 0
    optimizer.zero_grad()#清空上一步的梯度
    idx = np.random.randint(x.shape[0], size=batch_size)
    batch_cpu = Variable(torch.from_numpy(x[idx])).float()
    batch = batch_cpu.cuda()#很重要

    target_cpu = Variable(torch.from_numpy(y[idx])).float()
    target = target_cpu.cuda()#很重要
    output = model.forward(batch)
    loss += criterion(output, target)
    loss.backward()
    optimizer.step()

    if (i +1)%10 == 0:
        print('Loss at epoch[%s]: %.3f' % (i, loss.data[0]))
    losses.append(loss.data[0])

plt.plot(losses, '-or')
plt.xlabel("Epoch")
plt.xlabel("Loss")

plt.show()
```

综上，一个要在GPU上训练一个模型必须做到以下几点：
- 数据cuda化，包括src和tgt数据，不确定Variable包装时间是否有影响
- model cuda化，主要是里面进行矩阵运算的参数cuda化,但使用model.cuda()的时候，受影响的parameter其实只有nn中的哪些参数，所以如果你有其他的，自定义的参数也要放到GPU上预算，应该给这些参数特别的说明？？？？
- loss cuda化
- optimizer好像没关系
- 不知道在显卡上调用Variable.data后再转回Variable是否会影响他们的grad_func属性？？？